In [1]:
import datetime as dt
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import matplotlib.dates as mdates
from pylab import rcParams
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import mplfinance as mpf
import bs4 as bs
import pickle
import requests
import os
from collections import Counter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

test_size = 0.2
valid_size = 0.2
N = 21

n_estimators = 100
max_depth = 3
learning_rate = 0.1
min_child_weight = 1
subsample = 1
colsample_bytree = 1
colsample_bylevel = 1
gamma = 0
model_seed = 100

style.use('seaborn-darkgrid')

c:\python38\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
def get_mov_avg_std(df, col):
    """
    Get the mean and standard deviation of a data frame at a specific timestamp using values from t-1, t-2,..., t-N
    """

    mean_list = df[col].rolling(window = 5, min_periods = 1).mean()
    std_list = df[col].rolling(window = 5, min_periods = 1).std()

    mean_list = np.concatenate((np.array([np.nan]), np.array(mean_list[:-1])))
    std_list = np.concatenate((np.array([np.nan]), np.array(std_list[:-1])))

    df[col + '_MEAN'] = mean_list
    df[col + '_STD'] = std_list

    return df

def scale_row(row, feat_mean, feat_std):
    """
    Given a row of a data frame, scale it to having a mean value of 0 and variance of 1 using feat_mean and feat_std
    """

    feat_std = 0.001 if feat_std == 0 else feat_std

    row_scaled = (row-feat_mean)/feat_std
    
    return row_scaled

def train_pred_eval_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test, col_mean, col_std, seed = 100, n_estimators = 100, max_depth = 3, learning_rate = 0.1, min_child_weight = 1, subsample = 1, colsample_bytree = 1, colsample_bylevel = 1, gamma = 0):
    """
    Train the model, predict using the model, and evaluate the model for tuning purposes
    """

    model = XGBRegressor(seed = 100, n_estimators = n_estimators, max_depth = max_depth, learning_rate = learning_rate, min_child_weight = min_child_weight, subsample = subsample, colsample_bytree = colsample_bytree, colsample_bylevel = colsample_bylevel, gamma = gamma)

    model.fit(X_train_scaled, y_train_scaled)

    predict_scaled = model.predict(X_test_scaled)
    predict = predict_scaled * col_std + col_mean

    mse = mean_squared_error(y_test, predict)
    mae = mean_absolute_error(y_test, predict)

    return mse, mae, predict

In [3]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    ticker = ticker[:-1]
    tickers.append(ticker)
for n, i in enumerate(tickers):
    if i == 'BRK.B':
        tickers[n] = 'BRKB'
    elif i == 'BF.B':
        tickers[n] = 'BFB'

#with open('sp500tickers.pickle', 'wb') as f:
#    pickle.dump(tickers, f)

tickers

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRKB',
 'BBY',
 'BIO',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BFB',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO',


In [5]:
#with open('sp500tickers.pickle', 'rb') as f:
#        tickers = pickle.load(f)
if not os.path.exists('stock_dfs'):
    os.makedirs('stock_dfs')
start = dt.datetime(2010, 1, 1)
end = dt.datetime.now()
for ticker in tickers:
    symbols = '_,.'
    for c in symbols:
        if c in ticker:
            ticker.replace(c, '')
    if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
        df = web.DataReader(ticker, 'yahoo', start, end)#, api_key='RC3GG4A8WDBBST0I')
        df. reset_index(inplace = True)
        df.to_csv('stock_dfs/{}.csv'.format(ticker))
    else:
        print('Already have {}'.format(ticker))
        df = web.DataReader(ticker, 'yahoo', start, end)#, api_key='RC3GG4A8WDBBST0I')
        df. reset_index(inplace = True)
        df.to_csv('stock_dfs/{}.csv'.format(ticker))
        print('Updated {} file'.format(ticker))
    time.sleep(1)

Already have MMM
Updated MMM file
Already have ABT
Updated ABT file
Already have ABBV
Updated ABBV file
Already have ABMD
Updated ABMD file
Already have ACN
Updated ACN file
Already have ATVI
Updated ATVI file
Already have ADBE
Updated ADBE file
Already have AMD
Updated AMD file
Already have AAP
Updated AAP file
Already have AES
Updated AES file
Already have AFL
Updated AFL file
Already have A
Updated A file
Already have APD
Updated APD file
Already have AKAM
Updated AKAM file
Already have ALK
Updated ALK file
Already have ALB
Updated ALB file
Already have ARE
Updated ARE file
Already have ALXN
Updated ALXN file
Already have ALGN
Updated ALGN file
Already have ALLE
Updated ALLE file
Already have LNT
Updated LNT file
Already have ALL
Updated ALL file
Already have GOOGL
Updated GOOGL file
Already have GOOG
Updated GOOG file
Already have MO
Updated MO file
Already have AMZN
Updated AMZN file
Already have AMCR
Updated AMCR file
Already have AEE
Updated AEE file
Already have AAL
Updated AAL

Already have HPQ
Updated HPQ file
Already have HUM
Updated HUM file
Already have HBAN
Updated HBAN file
Already have HII
Updated HII file
Already have IEX
Updated IEX file
Already have IDXX
Updated IDXX file
Already have INFO
Updated INFO file
Already have ITW
Updated ITW file
Already have ILMN
Updated ILMN file
Already have INCY
Updated INCY file
Already have IR
Updated IR file
Already have INTC
Updated INTC file
Already have ICE
Updated ICE file
Already have IBM
Updated IBM file
Already have IP
Updated IP file
Already have IPG
Updated IPG file
Already have IFF
Updated IFF file
Already have INTU
Updated INTU file
Already have ISRG
Updated ISRG file
Already have IVZ
Updated IVZ file
Already have IPGP
Updated IPGP file
Already have IQV
Updated IQV file
Already have IRM
Updated IRM file
Already have JKHY
Updated JKHY file
Already have J
Updated J file
Already have JBHT
Updated JBHT file
Already have SJM
Updated SJM file
Already have JNJ
Updated JNJ file
Already have JCI
Updated JCI file


Updated WAB file
Already have WMT
Updated WMT file
Already have WBA
Updated WBA file
Already have DIS
Updated DIS file
Already have WM
Updated WM file
Already have WAT
Updated WAT file
Already have WEC
Updated WEC file
Already have WFC
Updated WFC file
Already have WELL
Updated WELL file
Already have WST
Updated WST file
Already have WDC
Updated WDC file
Already have WU
Updated WU file
Already have WRK
Updated WRK file
Already have WY
Updated WY file
Already have WHR
Updated WHR file
Already have WMB
Updated WMB file
Already have WLTW
Updated WLTW file
Already have WYNN
Updated WYNN file
Already have XEL
Updated XEL file
Already have XRX
Updated XRX file
Already have XLNX
Updated XLNX file
Already have XYL
Updated XYL file
Already have YUM
Updated YUM file
Already have ZBRA
Updated ZBRA file
Already have ZBH
Updated ZBH file
Already have ZION
Updated ZION file
Already have ZTS
Updated ZTS file


In [6]:
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2013-02-01,31.74,30.469999,31.50,31.010000,66789100.0,29.292076
1,2013-02-04,31.99,30.760000,31.09,31.020000,7695400.0,29.301533
2,2013-02-05,31.98,30.850000,31.25,31.040001,5013200.0,29.320419
3,2013-02-06,31.43,30.750000,30.98,31.030001,2126100.0,29.310972
4,2013-02-07,32.73,31.000000,31.00,32.000000,3800800.0,30.227238


In [11]:
with open('sp500tickers.pickle', 'rb') as f:
    tickers = pickle.load(f)

main_df = pd.DataFrame()

for count,ticker in enumerate(tickers):
    df = pd.read_csv('stock_dfs/{}.csv'.format(ticker), index_col = 0)
    df['{}_HL_PCT_DIFF'.format(ticker)] = (df['High'] - df['Low']) / df['Low']
    df['{}_PCT_CHNG'.format(ticker)] = (df['Close'] - df['Open']) / df['Open']
    df.rename(columns={'Adj Close':ticker}, inplace = True)
    df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis = 1, inplace = True)

    if main_df.empty:
        main_df = df
    else:
        main_df = main_df.merge(df, how = 'outer')
    if count % 10 == 0:
        print(count)

main_df.to_csv('sp500_joined_closes.csv')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [12]:
main_df

,Date,MMM,MMM_HL_PCT_DIFF,MMM_PCT_CHNG,ABT,ABT_HL_PCT_DIFF,ABT_PCT_CHNG,ABBV,ABBV_HL_PCT_DIFF,ABBV_PCT_CHNG,...,ZBRA_PCT_CHNG,ZBH,ZBH_HL_PCT_DIFF,ZBH_PCT_CHNG,ZION,ZION_HL_PCT_DIFF,ZION_PCT_CHNG,ZTS,ZTS_HL_PCT_DIFF,ZTS_PCT_CHNG
0,2010-01-04,62.327927,0.009435,-0.000842,18.082745,0.011869,0.004982,NaN,NaN,NaN,...,0.007025,55.568928,0.013994,0.011971,11.827960,0.038043,0.030139,NaN,NaN,NaN
1,2010-01-05,61.937550,0.018727,-0.003623,17.936651,0.013395,-0.008261,NaN,NaN,NaN,...,0.002452,57.328018,0.021929,0.009785,12.244997,0.045352,0.039157,NaN,NaN,NaN
2,2010-01-06,62.815929,0.013052,-0.002504,18.036259,0.010028,0.007045,NaN,NaN,NaN,...,0.036118,57.309509,0.010574,-0.002578,13.309779,0.098829,0.097293,NaN,NaN,NaN
3,2010-01-07,62.860950,0.019971,0.004921,18.185675,0.013131,0.008470,NaN,NaN,NaN,...,-0.019129,58.624191,0.024775,0.007478,14.800472,0.137124,0.114228,NaN,NaN,NaN
4,2010-01-08,63.303928,0.012245,0.007528,18.278645,0.010424,0.005296,NaN,NaN,NaN,...,0.000725,57.392849,0.028478,0.002912,14.560900,0.073454,0.013589,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2677,2020-08-21,161.710007,0.012131,0.004847,102.400002,0.016858,0.007577,94.860001,0.010602,-0.003571,...,-0.007498,136.160004,0.017463,-0.004533,31.590000,0.031918,-0.015274,158.369995,0.012914,-0.002457
2678,2020-08-24,163.529999,0.017375,0.012256,101.389999,0.025124,-0.012756,94.510002,0.011255,-0.004110,...,-0.014246,136.809998,0.024320,-0.002261,33.040001,0.048193,0.041943,156.759995,0.022662,-0.018717
2679,2020-08-25,163.100006,0.017424,-0.010676,102.470001,0.013593,0.004805,94.059998,0.009371,-0.001592,...,0.005937,137.600006,0.015128,-0.004846,33.060001,0.039336,-0.011364,158.580002,0.014236,0.003734
2680,2020-08-26,163.899994,0.019049,0.006324,103.190002,0.020047,0.011766,94.349998,0.015817,0.009091,...,0.023471,136.830002,0.013988,-0.001824,32.279999,0.027890,-0.026831,159.759995,0.020290,0.013834


In [13]:
main_df.isnull().sum()

Date                  0
MMM                   0
MMM_HL_PCT_DIFF       0
MMM_PCT_CHNG          0
ABT                   0
                   ... 
ZION_HL_PCT_DIFF      0
ZION_PCT_CHNG         0
ZTS                 775
ZTS_HL_PCT_DIFF     775
ZTS_PCT_CHNG        775
Length: 1516, dtype: int64